# Chapter 20 - 속성 디스크립터

## 오버라이딩 디스크립터와 논오버라이딩 디스크립터

파이썬이 속성을 처리하는 방식에는 커다란 비대칭성이 있음을 주의하라.<br>
일반적으로 객체를 통해 속성을 읽으면 객체에 정의된 속성을 반환하지만, 객체에 그 속성이 없으면 클래스 속성을 읽는다.<br>
한편 일반적으로 객체의 속성에 값을 할당하면 객체 안에 그 속성을 만들고 클래스에는 전혀 영향을 미치지 않는다.<br>
<br>
이런 비대칭성은 디스크립터에도 영향을 미쳐 \_\_set\_\_() 메서드의 정의 여부에 따라 두 가지 범주의 디스크립터를 생성한다.<br>
다음은 디스크립터 오버라이딩 동작을 살펴보기 위한 간단한 클래스이다.

In [2]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '<{} object>'.format(cls_name(obj))
    
def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))
    
### 이 예제에서 중요한 클래스 ###

class Overriding:
    """데이터 디스크립터 혹은 강제 디스크립터라고도 한다."""
    
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)
        
    def __set__(self, instance, value):
        print_args('set', self, instance, value)
        
class OverridingNoGet:
    """``__get__()``이 없는 오버라이딩 디스크립터"""
    
    def __set__(self, instance, value):
        print_args('set', self, instance, value)
        
class NonOverriding:
    """비데이터 디스크립터 혹은 가릴 수 있는 디스크립터라고도 한다."""
    
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)
        
class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()
    
    def spam(self):
        print('-> Managed.spam({})'.format(display(self)))

## 오버라이딩 디스크립터

\_\_set\_\_() 메서드를 구현하는 디스크립터를 **오버라이딩 디스크립터**라고 한다.<br>
비록 클래스 속성이기는 하지만, \_\_set\_\_() 메서드를 구현하는 디스크립터는 객체 속성에 할당하려는 시도를 가로채기 때문이다.<br>
프로퍼티도 오버라이딩 디스크립터라고 할 수 있다. 세터 함수를 제공하지 않더라도, property 클래스에서 기본적으로 제공하는 \_\_set\_\_() 메서드가 읽기 전용 속성임을 알려주기 위해 AttributeError를 발생시킨다.

In [3]:
obj = Managed()
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [4]:
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [5]:
obj.over = 7

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [7]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [9]:
obj.__dict__['over'] = 8
vars(obj)

{'over': 8}

In [10]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


## \_\_get\_\_()이 없는 오버라이딩 디스크립터

일반적으로 오버라이딩 디스크립터는 \_\_set\_\_()과 \_\_get\_\_() 메서드를 모두 구현하지만, \_\_set\_\_() 메서드만 오버라이드할 수도 있다.<br>
이때는 저장 연산만 디스크립터가 처리한다. 객체를 통해 디스크립터를 확인해보면 읽기 접근을 처리하는 \_\_get\_\_() 메서드가 없으므로 디스크립터 객체 자체가 반환된다.

In [11]:
obj.over_no_get

In [12]:
Managed.over_no_get

In [13]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [14]:
obj.over_no_get

In [15]:
obj.__dict__['over_no_get'] = 9
obj.over_no_get

9

In [16]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [17]:
obj.over_no_get

9

## 디스크립터 사용에 대한 조언

- 코드를 간결하게 작성하기 위해 프로퍼티를 사용하라
- 읽기 전용 디스크립터는 \_\_set\_\_()을 구현해야 한다
- 검증 디스크립터는 \_\_set\_\_()만 사용할 수 있다
- 캐시는 \_\_get\_\_()에서만 효율적으로 구현할 수 있다
- 특별 메서드 이외의 메서드는 객체 속성에 의해 가려질 수 있다